In [33]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import tempfile
from six.moves.urllib.request import urlopen
from six import BytesIO
import numpy as np
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps
import time
print(tf.__version__)

2.4.0


In [32]:
%%capture
%cd /content/drive/MyDrive/RohithWorkspace/Object-Detection-Website
!git add -A
!git commit -m "imported important libraries"
!git push